In [1]:
import pulp
import pandas as pd

In [2]:
model = pulp.LpProblem("Transportation_Cost_Minimization", pulp.LpMinimize)

In [3]:
supply = {'Factory_A': 35,'Factory_B': 50}
demand = {'Warehouse_1': 30,'Warehouse_2': 25,'Warehouse_3': 30}
costs = {('Factory_A', 'Warehouse_1'): 2,('Factory_A', 'Warehouse_2'): 4,('Factory_A', 'Warehouse_3'): 5,('Factory_B', 'Warehouse_1'): 3,('Factory_B', 
'Warehouse_2'): 1,('Factory_B', 'Warehouse_3'): 7}

In [4]:
routes = pulp.LpVariable.dicts("Route", (supply.keys(), demand.keys()), lowBound=0, cat='Continuous')

In [5]:
model += pulp.lpSum([routes[f][w] * costs[(f, w)] for f in supply for w in demand]), "Total_Transportation_Cost"

In [6]:
for f in supply:
    model += pulp.lpSum([routes[f][w] for w in demand]) <= supply[f], f"Supply_Constraint_{f}"

for w in demand:
    model += pulp.lpSum([routes[f][w] for f in supply]) >= demand[w], f"Demand_Constraint_{w}"

In [7]:
model.solve()
print("Status:", pulp.LpStatus[model.status])

Status: Optimal


In [8]:
for f in supply:
    for w in demand:
        print(f"Ship from {f} to {w}: {routes[f][w].varValue} units")
        
print("\nTotal Transportation Cost = ₹", pulp.value(model.objective))

Ship from Factory_A to Warehouse_1: 5.0 units
Ship from Factory_A to Warehouse_2: 0.0 units
Ship from Factory_A to Warehouse_3: 30.0 units
Ship from Factory_B to Warehouse_1: 25.0 units
Ship from Factory_B to Warehouse_2: 25.0 units
Ship from Factory_B to Warehouse_3: 0.0 units

Total Transportation Cost = ₹ 260.0


In [9]:
results = []
for f in supply:
    for w in demand:
        results.append([f, w, routes[f][w].varValue, costs[(f, w)]])

In [10]:
df = pd.DataFrame(results, columns=["Factory", "Warehouse", "Units Shipped", "Cost per Unit"])
df["Total Cost"] = df["Units Shipped"] * df["Cost per Unit"]

df

,Factory,Warehouse,Units Shipped,Cost per Unit,Total Cost
0,Factory_A,Warehouse_1,5.0,2,10.0
1,Factory_A,Warehouse_2,0.0,4,0.0
2,Factory_A,Warehouse_3,30.0,5,150.0
3,Factory_B,Warehouse_1,25.0,3,75.0
4,Factory_B,Warehouse_2,25.0,1,25.0
5,Factory_B,Warehouse_3,0.0,7,0.0


In [13]:
supply_df = pd.DataFrame.from_dict(supply, orient='index', columns=["Available Supply"])
demand_df = pd.DataFrame.from_dict(demand, orient='index', columns=["Required Demand"])

print("Supply:")
display(supply_df)

print("\nDemand:")
display(demand_df)

Supply:


,Available Supply
Factory_A,35
Factory_B,50



Demand:


,Required Demand
Warehouse_1,30
Warehouse_2,25
Warehouse_3,30


In [15]:
cost_df = pd.DataFrame(index=supply.keys(), columns=demand.keys())

for (f, w), c in costs.items():
    cost_df.loc[f, w] = c

cost_df = cost_df.astype(float)
cost_df.index.name = "Factory"

print("Transportation Cost Matrix (Rs. per unit):")
display(cost_df)

Transportation Cost Matrix (Rs. per unit):


,Warehouse_1,Warehouse_2,Warehouse_3
Factory,,,
Factory_A,2.0,4.0,5.0
Factory_B,3.0,1.0,7.0
